In [79]:
from functools import reduce
import json
import os
import operator
import pandas as pd
import sys

module_path = os.path.abspath(os.path.join('../src'))
if not module_path in sys.path:
    sys.path.insert(0, module_path)

from innoprod.sheet_tools import get_sheet_dfs
from innoprod.wrangling.wrangling_tools import characterise_df_columnwise, is_non_empty
from innoprod.wrangling.msyh_data_sharing import wrangle_roadmaps, wrangle_grants

In [4]:
data = get_sheet_dfs()
roadmaps_df = wrangle_roadmaps(data['Roadmaps'])
grants_df = wrangle_grants(data['Grants'])

# Basic Data Characterisation

In [5]:
roadmaps_characterisation = characterise_df_columnwise(roadmaps_df)
roadmaps_characterisation.to_csv('characterisation_roadmaps.csv')

In [6]:
grants_characterisation = characterise_df_columnwise(grants_df)
grants_characterisation.to_csv('characterisation_grants.csv')

In [7]:
df = data['Roadmaps']
col = 'Number of PT employees'
mask = (df[col] == '') == (df[col].isna())
df[mask][col].unique().size

np.int64(2)

## Cross-referencing Roadmaps and Grants
The two lists of **Client ID**s overlap perfectly:

In [8]:
set(grants_df['Client ID']) == set(roadmaps_df['Client ID'])

True

## Do withdrawn grant applications mean clients pull out completely?
This many clients withdrew from the scheme all together:

In [9]:
mask = grants_df['Application Status'] != 'Withdrew'

len(set(roadmaps_df['Client ID']).difference(set(grants_df[mask]['Client ID'])))

10

# Does Roadmaps > Number of GAFs match up with the data on Grants?

In [10]:
sum(roadmaps_df['Number of GAFs']) == grants_df['Grant ID'].count()

np.True_

# How many Roadmaps rows have all core data?

In [11]:
core_cols = [
    'Client ID',
    'Number of GAFs',
    'Primary_contact_id',
    'Nature of Business/core activity',
    'Turnover',
    'Enquiry Date',
    'Org Size by Number of FTE (calc)',
    'Number of FTE Employees (calc)',
    'Summary review of Edge Digital diagnostic report & current state and key improvement areas',
    'Current Digital Readiness Score (refer to PAS:1040)',
    'What are the internal barriers to growth? How do you intend to finance future growth? Are there sufficient leadership and management skills in the business to achieve your growth? What opportunities do you have to expand into new markets?',
    'Main historical barrier',
    'Details of any existing Digital Strategy',
    'Do you have a Digital Champion in place?',
    'Level of current Strategic Digital Skills/knowledge in the business',
    'Level of current Technical Digital Skills/knowledge in the business',
    'Whether the business is already investing/adopting/utilising Industry 4.0 Technologies, with examples',
    'Summary of the identified problems, including Gap Analysis',
    'Key potential industry 4.0 solutions to address the identified problems/gaps',
    'Recommended Action Plan to utilise Industry 4.0 Technology',
    'Overview of qualitative benefits of recommended Action Plan (positive/negative)',
    'Skills and other requirements that will be needed to successfully implement the recommended Action Plan',
    'Application area of technology in the Action Plan'
]

reqs_plans = [
   'Requirements/Plans: Short Term',
   'Requirements/Plans: Medium Term', 
   'Requirements/Plans: Long Term'
]

In [12]:
core_mask = [is_non_empty(roadmaps_df[col]) for col in core_cols]
core_mask = reduce(operator.and_, core_mask)

reqs_mask = [is_non_empty(roadmaps_df[col]) for col in reqs_plans]
reqs_mask = reduce(operator.or_, reqs_mask)

core_mask = core_mask & reqs_mask
sum(core_mask)

np.int64(90)

# Counting clients by size
* All clients
* Only those clients with values in all the core columns 

In [13]:
roadmaps_df.groupby('Org Size by Number of FTE (calc)').size()

Org Size by Number of FTE (calc)
                     7
Medium - 50-249     43
Micro - 1-9         60
Small - 10-49      110
dtype: int64

In [14]:
roadmaps_df[core_mask].groupby('Org Size by Number of FTE (calc)').size()

Org Size by Number of FTE (calc)
Medium - 50-249    17
Micro - 1-9        27
Small - 10-49      46
dtype: int64

## Analysis of zeros in FTE Employees

In [15]:
zero_fte_mask = (roadmaps_df['Number of FTE Employees (calc)'] == 0).fillna(False)
sum(zero_fte_mask)
# zero_fte_mask

np.int64(81)

In [16]:
nan_fte_mask = roadmaps_df['Number of FTE Employees (calc)'].isna()
sum(nan_fte_mask)

7

In [17]:
roadmaps_df[zero_fte_mask].groupby('Org Size by Number of FTE (calc)').size()

Org Size by Number of FTE (calc)
Medium - 50-249    15
Micro - 1-9        24
Small - 10-49      42
dtype: int64

In [18]:
roadmaps_df[zero_fte_mask][['Org Size by Number of FTE (calc)', 'Number of FT employees', 'Number of PT employees', 'Willing to be approached for case study?']]

,Org Size by Number of FTE (calc),Number of FT employees,Number of PT employees,Willing to be approached for case study?
0,Micro - 1-9,<NA>,<NA>,No
8,Small - 10-49,<NA>,<NA>,No
16,Small - 10-49,<NA>,<NA>,NaN
19,Medium - 50-249,<NA>,<NA>,NaN
21,Small - 10-49,<NA>,<NA>,
...,...,...,...,...
208,Small - 10-49,<NA>,<NA>,Yes
211,Small - 10-49,<NA>,<NA>,NaN
212,Micro - 1-9,<NA>,<NA>,Yes
216,Medium - 50-249,<NA>,<NA>,Yes


In [19]:
roadmaps_df[nan_fte_mask][['Org Size by Number of FTE (calc)', 'Number of FT employees', 'Number of PT employees', 'Willing to be approached for case study?']]

,Org Size by Number of FTE (calc),Number of FT employees,Number of PT employees,Willing to be approached for case study?
3,,<NA>,<NA>,NaN
75,,<NA>,<NA>,No
82,,<NA>,<NA>,Yes
156,,<NA>,<NA>,Yes
173,,<NA>,<NA>,
177,,<NA>,<NA>,No
179,,<NA>,<NA>,No


# Willing to be approached for case study

In [20]:
willing_mask = (roadmaps_df['Willing to be approached for case study?'] == 'Yes')

In [21]:
sum(willing_mask)

90

In [22]:
sum(willing_mask & core_mask)

np.int64(38)

In [23]:
roadmaps_df[willing_mask].groupby('Org Size by Number of FTE (calc)').size()

Org Size by Number of FTE (calc)
                    2
Medium - 50-249    23
Micro - 1-9        21
Small - 10-49      44
dtype: int64

In [24]:
roadmaps_df[willing_mask & core_mask].groupby('Org Size by Number of FTE (calc)').size()

Org Size by Number of FTE (calc)
Medium - 50-249     7
Micro - 1-9        10
Small - 10-49      21
dtype: int64

# Grants data

In [25]:
grants_df.groupby(['Application Status', 'Claim Status']).size()

Application Status  Claim Status
                    Defrayed         44
Defrayed            Defrayed        183
Engaged             Defrayed         51
Offered             Defrayed         12
Withdrew            Withdrawn        23
dtype: int64